# Import the libary

In [386]:
from ast import literal_eval
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import pandas as pd
import sidetable

# download nltk packages
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

# Data preparation

In [387]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("damienbeneschi/krakow-ta-restaurans-data-raw")

# print("Path to dataset files:", path)

In [388]:
df = pd.read_csv('/Users/remaalnssiry/.cache/kagglehub/datasets/damienbeneschi/krakow-ta-restaurans-data-raw/versions/5/TA_restaurants_curated.csv')
df


,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959
3,3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",/Restaurant_Review-g188590-d1239229-Reviews-Vi...,d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",/Restaurant_Review-g188590-d6864170-Reviews-Li...,d6864170
...,...,...,...,...,...,...,...,...,...,...,...
125522,1662,Konrad Kaffee- & Cocktailbar,Zurich,NaN,NaN,NaN,NaN,NaN,NaN,/Restaurant_Review-g188113-d13273526-Reviews-K...,d13273526
125523,1663,Blueberry American Bakery,Zurich,['Cafe'],NaN,NaN,NaN,NaN,NaN,/Restaurant_Review-g188113-d13292844-Reviews-B...,d13292844
125524,1664,Restaurant Bahnhof,Zurich,NaN,NaN,NaN,NaN,NaN,NaN,/Restaurant_Review-g188113-d13296092-Reviews-R...,d13296092
125525,1665,Yoyo Pizza,Zurich,['Fast Food'],NaN,NaN,NaN,NaN,NaN,/Restaurant_Review-g188113-d13323362-Reviews-Y...,d13323362


In [389]:
# Load JSON data from file (for multiple restaurants)
file_path = "/Users/remaalnssiry/code/Remafsa/geo-pic/geo-pic/data/restaurant_data.json"
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert JSON to DataFrame (multiple entries)
df_saudi= pd.DataFrame(data)


# sprate resturant name and city 

In [390]:
valid_cities = [
    'Riyadh', 'Jeddah', 'Makkah', 'Madinah', 'Khobar',
    'Dammam', 'Taif', 'Jazan', 'AlUla', 'Abha',
    'Dhahran', 'Al Khobar', 'Buraidah', 'Hafar Al-Batin',
    'Najran', 'Al Jubail', 'Khamis Mushait', 'Tabuk',
    'Yanbu', 'Arar', 'Sakakah', 'Al-Qunfudhah', 'Al Bahah',
    'Al Hufuf', 'Al-Mubarraz', 'Al-Faisaliyah', 'Qatif',
    'Wadi Al Dawasir', 'Al Majma’ah', 'Al-‘Ula', 'Al Rass',
    'Al Namas', 'Al Qassim', 'Aseer', 'Dawadmi', 'Riyadh Region',
    'Eastern Province', 'Southern Province', "Al Ula",
]
restaurant_city_map = {
    "The Quarter Restaurant": "Jeddah",
    "Café Bateel - Middle Eastern Boutique Café": "Riyadh",
    "El Panero Fine Dining Restaurant": "Al Khobar",
    "Early Club - Breakfast & Brunch": "Jeddah",
    "Dakatna Cafe & Coffee House": "Makkah",
    "Meraki Greek Restaurant - Al Faialiah Hotel - RIyadh": "Riyadh",
    "Al Fanar Restaurant and Cafe - Roshn Front": "Riyadh",
    "Davanti Lounge - دافانتي لاونج": "Madinah",
    "Shehrazad Indian Restaurant - المطعم الهندي شهرزاد": "Jeddah"
}

def find_city(restaurant_name):
    for city in valid_cities:
        if city in restaurant_name:
            return city
    return None  # Return None if no city is found

# Apply the function to create a new 'city' column
df_saudi['city'] = df_saudi['restaurant_name'].apply(find_city)

# Use restaurant_city_map where city is still None
df_saudi['city'] = df_saudi['city'].combine_first(df_saudi['restaurant_name'].map(restaurant_city_map))


In [391]:
df_saudi['restaurant_name'] = df_saudi['restaurant_name'].str.split(' - ').str[0]

In [392]:
df_saudi

,restaurant_name,description,rate,reviews,cuisines,restaurant_type,restaurant_amenities,location,images,menu,url,city
0,Thaiya,[Thaiya is a charming Thai restaurant in Riyad...,5.0,[Creative take on Thai cuisine. Not very tradi...,[Thai],[Fine Dining],"[Air Conditioning, Parking]","Thaiya - تايا, Prince Muhammad Ibn Abd Al Aziz...",[https://welcomesaudi.com/uploads/0000/64/2024...,https://qr.finedinemenu.com/thaiya,https://welcomesaudi.com/restaurant/thaiya-riyadh,Riyadh
1,Zafran Valley,[Zafran Valley is a premium dining destination...,5.0,[Loved the delicious food at this lovely resta...,[Indian],[],[],"Zafran Valley Restaurant مطعم وادي زعفران, Hai...",[https://welcomesaudi.com/uploads/0000/hotels/...,None,https://welcomesaudi.com/restaurant/zafran-val...,Riyadh
2,Okla,"[Mutah Beale, a name hip-hop enthusiasts might...",5.0,"[If you want a burger done right, this is the ...",[Armenian],[Burger],"[Air Conditioning, Outdoor Seating, Parking]","OKLA، Musa Ibn Nusair Street, Riyadh Saudi Arabia",[https://welcomesaudi.com/uploads/0000/64/2024...,https://www.instagram.com/stories/highlights/1...,https://welcomesaudi.com/restaurant/okla-riyad...,Riyadh
3,Gloria Cafe,"[Gloria Café serves hot and cold drinks, fresh...",4.7,"[Quiet and comfortable cafe?, It's a perfect c...",[],"[Breakfast, Cafe, Coffee & Tea, Desserts, Fast...","[Air Conditioning, Family Seating, Outdoor Sea...","Gloria Cafe, قاسم زينة، Jeddah Saudi Arabia",[https://welcomesaudi.com/uploads/0000/64/2021...,#,https://welcomesaudi.com/restaurant/gloria-caf...,Jeddah
4,Nobu,[Nobu Jeddah is the finest Japanese Restaurant...,4.7,[- The restaurant is too expensive and it’s no...,"[Japanese, Peruvian]","[Fine Dining, Seafood]","[Air Conditioning, Outdoor Seating, Parking, R...","نوبو، North next to Al Nawras Square, Al Kurna...",[https://welcomesaudi.com/uploads/0000/1/2023/...,https://qr1.mydigimenu.com/c04eeaa0-f83d-4bfd-...,https://welcomesaudi.com/restaurant/nobu-jedda...,Jeddah
...,...,...,...,...,...,...,...,...,...,...,...,...
1438,Al Sayyadiyah Resturant,[Al Sayyadiyah Restaurant is one of the finest...,3.2,[Very bad restaurant. No management. Take time...,[Arabic],"[Barbeque & Grill, Seafood, Takeout]","[Air Conditioning, Family Seating, Parking]","Alsayyadiyah Resturant, المكرونة، Jeddah Saudi...",[https://welcomesaudi.com/uploads/0000/64/2022...,#,https://welcomesaudi.com/restaurant/alsayyadiy...,Jeddah
1439,Al Sanabel Turkish Restaurant,[Al Sanabel restaurant is located in the hotel...,3.2,[Worst ever experience there no food no milk f...,"[Arabic, Middle Eastern, Turkish]","[Barbeque & Grill, Cafe, Desserts, Fine Dining...","[Air Conditioning, Family Seating]","Sanabel Restaurant, Bada'ah, Medina 42311, Sau...",[https://welcomesaudi.com/uploads/0000/64/2021...,#,https://welcomesaudi.com/restaurant/al-sanabel...,Madinah
1440,Lotus Restaurant,[Lotus Restaurant is one of the best restauran...,3.1,"[It’s a very nice hotel to stay at, offering s...",[International],"[Bakery, Barbeque & Grill, Breakfast, Coffee &...","[Air Conditioning, Family Seating, Parking]",Millennium Makkah Al Naseem Hotel فندق ميلينيو...,[https://welcomesaudi.com/uploads/0000/64/2022...,#,https://welcomesaudi.com/restaurant/lotus-rest...,Makkah
1441,La Brasserie,[La Brasserie is a modern multi-cuisine restau...,3.1,"[I stayed in this hotel more than 5 times, roo...",[International],"[Bakery, Barbeque & Grill, Breakfast, Cafe, Co...","[Air Conditioning, Family Seating, Parking, Wi...","Le Méridien Jeddah, Medinah Road, Ar Rawdah, J...",[https://welcomesaudi.com/uploads/0000/64/2021...,#,https://welcomesaudi.com/restaurant/la-brasser...,Jeddah


In [393]:
df_saudi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1443 entries, 0 to 1442
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   restaurant_name       1443 non-null   object
 1   description           1443 non-null   object
 2   rate                  1443 non-null   object
 3   reviews               1443 non-null   object
 4   cuisines              1443 non-null   object
 5   restaurant_type       1443 non-null   object
 6   restaurant_amenities  1443 non-null   object
 7   location              1443 non-null   object
 8   images                1443 non-null   object
 9   menu                  1423 non-null   object
 10  url                   1443 non-null   object
 11  city                  1443 non-null   object
dtypes: object(12)
memory usage: 135.4+ KB


In [394]:
df_saudi["restaurant_name"].unique()

array(['Thaiya', 'Zafran Valley', 'Okla', ..., 'Al Sayyadiyah Resturant',
       'Al Sanabel Turkish Restaurant', 'Park Inn By Radisson Restaurant'],
      dtype=object)

In [395]:
df_saudi['city'] = df_saudi['city'].replace({'Al Ula': 'AlUla'})


In [396]:
df_saudi.columns

Index(['restaurant_name', 'description', 'rate', 'reviews', 'cuisines',
       'restaurant_type', 'restaurant_amenities', 'location', 'images', 'menu',
       'url', 'city'],
      dtype='object')

In [397]:
df.columns

Index(['Unnamed: 0', 'Name', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA'],
      dtype='object')

In [398]:
df_saudi.head()

,restaurant_name,description,rate,reviews,cuisines,restaurant_type,restaurant_amenities,location,images,menu,url,city
0,Thaiya,[Thaiya is a charming Thai restaurant in Riyad...,5.0,[Creative take on Thai cuisine. Not very tradi...,[Thai],[Fine Dining],"[Air Conditioning, Parking]","Thaiya - تايا, Prince Muhammad Ibn Abd Al Aziz...",[https://welcomesaudi.com/uploads/0000/64/2024...,https://qr.finedinemenu.com/thaiya,https://welcomesaudi.com/restaurant/thaiya-riyadh,Riyadh
1,Zafran Valley,[Zafran Valley is a premium dining destination...,5.0,[Loved the delicious food at this lovely resta...,[Indian],[],[],"Zafran Valley Restaurant مطعم وادي زعفران, Hai...",[https://welcomesaudi.com/uploads/0000/hotels/...,None,https://welcomesaudi.com/restaurant/zafran-val...,Riyadh
2,Okla,"[Mutah Beale, a name hip-hop enthusiasts might...",5.0,"[If you want a burger done right, this is the ...",[Armenian],[Burger],"[Air Conditioning, Outdoor Seating, Parking]","OKLA، Musa Ibn Nusair Street, Riyadh Saudi Arabia",[https://welcomesaudi.com/uploads/0000/64/2024...,https://www.instagram.com/stories/highlights/1...,https://welcomesaudi.com/restaurant/okla-riyad...,Riyadh
3,Gloria Cafe,"[Gloria Café serves hot and cold drinks, fresh...",4.7,"[Quiet and comfortable cafe?, It's a perfect c...",[],"[Breakfast, Cafe, Coffee & Tea, Desserts, Fast...","[Air Conditioning, Family Seating, Outdoor Sea...","Gloria Cafe, قاسم زينة، Jeddah Saudi Arabia",[https://welcomesaudi.com/uploads/0000/64/2021...,#,https://welcomesaudi.com/restaurant/gloria-caf...,Jeddah
4,Nobu,[Nobu Jeddah is the finest Japanese Restaurant...,4.7,[- The restaurant is too expensive and it’s no...,"[Japanese, Peruvian]","[Fine Dining, Seafood]","[Air Conditioning, Outdoor Seating, Parking, R...","نوبو، North next to Al Nawras Square, Al Kurna...",[https://welcomesaudi.com/uploads/0000/1/2023/...,https://qr1.mydigimenu.com/c04eeaa0-f83d-4bfd-...,https://welcomesaudi.com/restaurant/nobu-jedda...,Jeddah


In [399]:
df_saudi.drop(columns=['location','rate','images', 'menu', 'url'], inplace=True)

In [400]:
df_saudi

,restaurant_name,description,reviews,cuisines,restaurant_type,restaurant_amenities,city
0,Thaiya,[Thaiya is a charming Thai restaurant in Riyad...,[Creative take on Thai cuisine. Not very tradi...,[Thai],[Fine Dining],"[Air Conditioning, Parking]",Riyadh
1,Zafran Valley,[Zafran Valley is a premium dining destination...,[Loved the delicious food at this lovely resta...,[Indian],[],[],Riyadh
2,Okla,"[Mutah Beale, a name hip-hop enthusiasts might...","[If you want a burger done right, this is the ...",[Armenian],[Burger],"[Air Conditioning, Outdoor Seating, Parking]",Riyadh
3,Gloria Cafe,"[Gloria Café serves hot and cold drinks, fresh...","[Quiet and comfortable cafe?, It's a perfect c...",[],"[Breakfast, Cafe, Coffee & Tea, Desserts, Fast...","[Air Conditioning, Family Seating, Outdoor Sea...",Jeddah
4,Nobu,[Nobu Jeddah is the finest Japanese Restaurant...,[- The restaurant is too expensive and it’s no...,"[Japanese, Peruvian]","[Fine Dining, Seafood]","[Air Conditioning, Outdoor Seating, Parking, R...",Jeddah
...,...,...,...,...,...,...,...
1438,Al Sayyadiyah Resturant,[Al Sayyadiyah Restaurant is one of the finest...,[Very bad restaurant. No management. Take time...,[Arabic],"[Barbeque & Grill, Seafood, Takeout]","[Air Conditioning, Family Seating, Parking]",Jeddah
1439,Al Sanabel Turkish Restaurant,[Al Sanabel restaurant is located in the hotel...,[Worst ever experience there no food no milk f...,"[Arabic, Middle Eastern, Turkish]","[Barbeque & Grill, Cafe, Desserts, Fine Dining...","[Air Conditioning, Family Seating]",Madinah
1440,Lotus Restaurant,[Lotus Restaurant is one of the best restauran...,"[It’s a very nice hotel to stay at, offering s...",[International],"[Bakery, Barbeque & Grill, Breakfast, Coffee &...","[Air Conditioning, Family Seating, Parking]",Makkah
1441,La Brasserie,[La Brasserie is a modern multi-cuisine restau...,"[I stayed in this hotel more than 5 times, roo...",[International],"[Bakery, Barbeque & Grill, Breakfast, Cafe, Co...","[Air Conditioning, Family Seating, Parking, Wi...",Jeddah


In [401]:
df_saudi.rename(columns={'restaurant_name': 'name', 'city': 'city', 'cuisines': 'style',
                    'reviews': 'reviews' }, inplace=True)

In [402]:
df_saudi

,name,description,reviews,style,restaurant_type,restaurant_amenities,city
0,Thaiya,[Thaiya is a charming Thai restaurant in Riyad...,[Creative take on Thai cuisine. Not very tradi...,[Thai],[Fine Dining],"[Air Conditioning, Parking]",Riyadh
1,Zafran Valley,[Zafran Valley is a premium dining destination...,[Loved the delicious food at this lovely resta...,[Indian],[],[],Riyadh
2,Okla,"[Mutah Beale, a name hip-hop enthusiasts might...","[If you want a burger done right, this is the ...",[Armenian],[Burger],"[Air Conditioning, Outdoor Seating, Parking]",Riyadh
3,Gloria Cafe,"[Gloria Café serves hot and cold drinks, fresh...","[Quiet and comfortable cafe?, It's a perfect c...",[],"[Breakfast, Cafe, Coffee & Tea, Desserts, Fast...","[Air Conditioning, Family Seating, Outdoor Sea...",Jeddah
4,Nobu,[Nobu Jeddah is the finest Japanese Restaurant...,[- The restaurant is too expensive and it’s no...,"[Japanese, Peruvian]","[Fine Dining, Seafood]","[Air Conditioning, Outdoor Seating, Parking, R...",Jeddah
...,...,...,...,...,...,...,...
1438,Al Sayyadiyah Resturant,[Al Sayyadiyah Restaurant is one of the finest...,[Very bad restaurant. No management. Take time...,[Arabic],"[Barbeque & Grill, Seafood, Takeout]","[Air Conditioning, Family Seating, Parking]",Jeddah
1439,Al Sanabel Turkish Restaurant,[Al Sanabel restaurant is located in the hotel...,[Worst ever experience there no food no milk f...,"[Arabic, Middle Eastern, Turkish]","[Barbeque & Grill, Cafe, Desserts, Fine Dining...","[Air Conditioning, Family Seating]",Madinah
1440,Lotus Restaurant,[Lotus Restaurant is one of the best restauran...,"[It’s a very nice hotel to stay at, offering s...",[International],"[Bakery, Barbeque & Grill, Breakfast, Coffee &...","[Air Conditioning, Family Seating, Parking]",Makkah
1441,La Brasserie,[La Brasserie is a modern multi-cuisine restau...,"[I stayed in this hotel more than 5 times, roo...",[International],"[Bakery, Barbeque & Grill, Breakfast, Cafe, Co...","[Air Conditioning, Family Seating, Parking, Wi...",Jeddah


In [403]:
def list_to_string(style_list):
    if isinstance(style_list, list):  # Check if it's a list
        return ', '.join(style_list)  # Join list elements with a comma
    return ''  # Return an empty string for non-list types

# Apply the cleaning function
df_saudi['style'] = df_saudi['style'].apply(list_to_string)
df_saudi['restaurant_type'] = df_saudi['restaurant_type'].apply(list_to_string)
df_saudi['restaurant_amenities'] = df_saudi['restaurant_amenities'].apply(list_to_string)
df_saudi['reviews'] = df_saudi['reviews'].apply(list_to_string)
df_saudi['description'] = df_saudi['description'].apply(list_to_string)


In [404]:
df_saudi

,name,description,reviews,style,restaurant_type,restaurant_amenities,city
0,Thaiya,Thaiya is a charming Thai restaurant in Riyadh...,Creative take on Thai cuisine. Not very tradit...,Thai,Fine Dining,"Air Conditioning, Parking",Riyadh
1,Zafran Valley,Zafran Valley is a premium dining destination ...,Loved the delicious food at this lovely restau...,Indian,,,Riyadh
2,Okla,"Mutah Beale, a name hip-hop enthusiasts might ...","If you want a burger done right, this is the s...",Armenian,Burger,"Air Conditioning, Outdoor Seating, Parking",Riyadh
3,Gloria Cafe,"Gloria Café serves hot and cold drinks, fresh ...","Quiet and comfortable cafe?, It's a perfect ca...",,"Breakfast, Cafe, Coffee & Tea, Desserts, Fast ...","Air Conditioning, Family Seating, Outdoor Seat...",Jeddah
4,Nobu,Nobu Jeddah is the finest Japanese Restaurant ...,- The restaurant is too expensive and it’s not...,"Japanese, Peruvian","Fine Dining, Seafood","Air Conditioning, Outdoor Seating, Parking, Ro...",Jeddah
...,...,...,...,...,...,...,...
1438,Al Sayyadiyah Resturant,Al Sayyadiyah Restaurant is one of the finest ...,Very bad restaurant. No management. Take time ...,Arabic,"Barbeque & Grill, Seafood, Takeout","Air Conditioning, Family Seating, Parking",Jeddah
1439,Al Sanabel Turkish Restaurant,Al Sanabel restaurant is located in the hotel ...,Worst ever experience there no food no milk fo...,"Arabic, Middle Eastern, Turkish","Barbeque & Grill, Cafe, Desserts, Fine Dining,...","Air Conditioning, Family Seating",Madinah
1440,Lotus Restaurant,Lotus Restaurant is one of the best restaurant...,"It’s a very nice hotel to stay at, offering sp...",International,"Bakery, Barbeque & Grill, Breakfast, Coffee & ...","Air Conditioning, Family Seating, Parking",Makkah
1441,La Brasserie,La Brasserie is a modern multi-cuisine restaur...,"I stayed in this hotel more than 5 times, room...",International,"Bakery, Barbeque & Grill, Breakfast, Cafe, Cof...","Air Conditioning, Family Seating, Parking, Wi-...",Jeddah


# Data reduction

In [405]:
# Drop unnecessary columns
df.drop(columns=['Unnamed: 0','URL_TA','ID_TA', 'Ranking', 'Number of Reviews',
                'Rating',"Price Range"], inplace=True)

# Rename columns for better convention
df.rename(columns={'Name': 'name', 'City': 'city', 'Cuisine Style': 'style', 'Reviews': 'reviews' }, inplace=True)

display('Dataset after attributes reduction and renaming', df.head())

'Dataset after attributes reduction and renaming'

,name,city,style,reviews
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']","[['Just like home', 'A Warm Welcome to Wintry ..."
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...","[['Great food and staff', 'just perfect'], ['0..."
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...","[['Satisfaction', 'Delicious old school restau..."
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...","[['True five star dinner', 'A superb evening o..."
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...","[['Best meal.... EVER', 'super food experience..."


In [406]:
display('Missing values', df.stb.missing())


'Missing values'

,missing,total,percent
style,31351,125527,24.975503
reviews,9616,125527,7.660503
name,0,125527,0.000000
city,0,125527,0.000000


# For simplification, I drop missing values

In [407]:
# Replace reviews that are empty to pandas NA
df['reviews'].replace("[[], []]", pd.NA, inplace=True)

print("Before drop: %s" % df.shape[0])
df.dropna(inplace=True)
print("After drop: %s" % df.shape[0])

Before drop: 125527
After drop: 79381


# Data transformation

In [408]:
df_saudi.columns

Index(['name', 'description', 'reviews', 'style', 'restaurant_type',
       'restaurant_amenities', 'city'],
      dtype='object')

In [409]:
df_saudi['reviews'] = df_saudi['reviews'].str.lower()
df_saudi['style'] = df_saudi['style'].str.lower()
df_saudi['city'] = df_saudi['city'].str.lower()
df_saudi['description'] = df_saudi['description'].str.lower()
df_saudi['restaurant_type'] = df_saudi['restaurant_type'].str.lower()
df_saudi['restaurant_amenities'] = df_saudi['restaurant_amenities'].str.lower()
display('Dataset after converting to lower case', df_saudi.head())

'Dataset after converting to lower case'

,name,description,reviews,style,restaurant_type,restaurant_amenities,city
0,Thaiya,thaiya is a charming thai restaurant in riyadh...,creative take on thai cuisine. not very tradit...,thai,fine dining,"air conditioning, parking",riyadh
1,Zafran Valley,zafran valley is a premium dining destination ...,loved the delicious food at this lovely restau...,indian,,,riyadh
2,Okla,"mutah beale, a name hip-hop enthusiasts might ...","if you want a burger done right, this is the s...",armenian,burger,"air conditioning, outdoor seating, parking",riyadh
3,Gloria Cafe,"gloria café serves hot and cold drinks, fresh ...","quiet and comfortable cafe?, it's a perfect ca...",,"breakfast, cafe, coffee & tea, desserts, fast ...","air conditioning, family seating, outdoor seat...",jeddah
4,Nobu,nobu jeddah is the finest japanese restaurant ...,- the restaurant is too expensive and it’s not...,"japanese, peruvian","fine dining, seafood","air conditioning, outdoor seating, parking, ro...",jeddah


In [410]:
df['reviews'] = df['reviews'].str.lower()
df['style'] = df['style'].str.lower()
df['city'] = df['city'].str.lower()
display('Dataset after converting to lower case', df.head())

'Dataset after converting to lower case'

,name,city,style,reviews
0,Martine of Martine's Table,amsterdam,"['french', 'dutch', 'european']","[['just like home', 'a warm welcome to wintry ..."
1,De Silveren Spiegel,amsterdam,"['dutch', 'european', 'vegetarian friendly', '...","[['great food and staff', 'just perfect'], ['0..."
2,La Rive,amsterdam,"['mediterranean', 'french', 'international', '...","[['satisfaction', 'delicious old school restau..."
3,Vinkeles,amsterdam,"['french', 'european', 'international', 'conte...","[['true five star dinner', 'a superb evening o..."
4,Librije's Zusje Amsterdam,amsterdam,"['dutch', 'european', 'international', 'vegeta...","[['best meal.... ever', 'super food experience..."


In [411]:
df['style'] = df['style'].apply(lambda x: literal_eval(x))

# Join the list items together
df['style'] = df['style'].apply(lambda x: ', '.join(x))

In [412]:
# For `reviews` column, we have some (5) dirty items that we need
# to handle explicitly and then remove them from dataset.
def convert_to_list(x):
    try:
        return literal_eval(x)
    except ValueError:
        return pd.NA

# Convert strings to python list and dirty data to pandas NA
df['reviews'] = df['reviews'].apply(convert_to_list)

# Drop dirty items
print("Before drop: %s" % df.shape[0])
df.dropna(inplace=True)
print("After drop: %s" % df.shape[0])

# Join the list items together, notice that we only send first item
# of list (reviews) to the function and ignore the second (dates)
df['reviews'] = df['reviews'].apply(lambda x: ', '.join(x[0]))

display('Dataset after we clean `reviews` and `style` columns', df.head())

Before drop: 79381
After drop: 79376


'Dataset after we clean `reviews` and `style` columns'

,name,city,style,reviews
0,Martine of Martine's Table,amsterdam,"french, dutch, european","just like home, a warm welcome to wintry amste..."
1,De Silveren Spiegel,amsterdam,"dutch, european, vegetarian friendly, gluten f...","great food and staff, just perfect"
2,La Rive,amsterdam,"mediterranean, french, international, european...","satisfaction, delicious old school restaurant"
3,Vinkeles,amsterdam,"french, european, international, contemporary,...","true five star dinner, a superb evening of fin..."
4,Librije's Zusje Amsterdam,amsterdam,"dutch, european, international, vegetarian fri...","best meal.... ever, super food experience"


# countries

In [413]:
word_cities_df = pd.read_csv('/Users/remaalnssiry/code/Remafsa/geo-pic/geo-pic/data/city_country_mapping.csv')
word_cities_df

,city,country
0,amsterdam,Netherlands
1,athens,Greece
2,barcelona,Spain
3,berlin,Germany
4,bratislava,Slovakia
5,brussels,Belgium
6,budapest,Hungary
7,copenhagen,Denmark
8,dublin,Ireland
9,edinburgh,Scotland


In [414]:
print('Number of cities: %s ' % df['city'].nunique())
cities_list = df['city'].unique()
print(cities_list)

Number of cities: 31 
['amsterdam' 'athens' 'barcelona' 'berlin' 'bratislava' 'brussels'
 'budapest' 'copenhagen' 'dublin' 'edinburgh' 'geneva' 'hamburg'
 'helsinki' 'krakow' 'lisbon' 'ljubljana' 'london' 'luxembourg' 'lyon'
 'madrid' 'milan' 'munich' 'oporto' 'oslo' 'paris' 'prague' 'rome'
 'stockholm' 'vienna' 'warsaw' 'zurich']


In [415]:

# Convert all attributes to lowercase:
word_cities_df['country'] = word_cities_df['country'].str.lower()
word_cities_df['city'] = word_cities_df['city'].str.lower()

display("Sample of word cities dataset", word_cities_df.head())

'Sample of word cities dataset'

,city,country
0,amsterdam,netherlands
1,athens,greece
2,barcelona,spain
3,berlin,germany
4,bratislava,slovakia


In [416]:
# Merge for adding country column to data frame
df = pd.merge(df, word_cities_df, on='city', how='inner')


# Reindex data frame to move country next to city
df = df.reindex(columns=['name', 'city', 'country', 'style', 'reviews'])

display("Sample of our new data frame with country column", df.sample(5))

'Sample of our new data frame with country column'

,name,city,country,style,reviews
71465,Cafe Tabac,stockholm,sweden,"bar, mediterranean, european, spanish, vegetar...","awseome tapas, trendy, but the food and servic..."
19391,Marcel's,dublin,ireland,"french, european, gluten free options, vegetar...","lovely food, sublime service, great christ...,..."
38278,Londinium,london,united kingdom,"european, british, vegetarian friendly","breakfast, tasty food service lacking"
8821,Bodega La Clau,barcelona,spain,"latin, mediterranean, spanish, chilean, south ...","must go!, perfect atmosphere and great wine"
69888,Gege 2,rome,italy,pizza,"mmmmmm good pizza, best pizza ever!"


In [417]:
print('Number of countries: %s ' % df['country'].nunique())
countries_list = df['country'].unique()
print(countries_list)

Number of countries: 23 
['netherlands' 'greece' 'spain' 'germany' 'slovakia' 'belgium' 'hungary'
 'denmark' 'ireland' 'scotland' 'switzerland' 'finland' 'portugal'
 'slovenia' 'united kingdom' 'luxembourg' 'france' 'italy' 'norway'
 'czech republic' 'sweden' 'austria' 'poland']


In [418]:
df_saudi = pd.merge(df_saudi, word_cities_df, on='city', how='inner')
df_saudi = df_saudi.reindex(columns=['name','city',"country", 'description', 'style', 'restaurant_type',
       'restaurant_amenities',"reviews"])
df_saudi


,name,city,country,description,style,restaurant_type,restaurant_amenities,reviews
0,Thaiya,riyadh,saudi arabia,thaiya is a charming thai restaurant in riyadh...,thai,fine dining,"air conditioning, parking",creative take on thai cuisine. not very tradit...
1,Zafran Valley,riyadh,saudi arabia,zafran valley is a premium dining destination ...,indian,,,loved the delicious food at this lovely restau...
2,Okla,riyadh,saudi arabia,"mutah beale, a name hip-hop enthusiasts might ...",armenian,burger,"air conditioning, outdoor seating, parking","if you want a burger done right, this is the s..."
3,Gloria Cafe,jeddah,saudi arabia,"gloria café serves hot and cold drinks, fresh ...",,"breakfast, cafe, coffee & tea, desserts, fast ...","air conditioning, family seating, outdoor seat...","quiet and comfortable cafe?, it's a perfect ca..."
4,Nobu,jeddah,saudi arabia,nobu jeddah is the finest japanese restaurant ...,"japanese, peruvian","fine dining, seafood","air conditioning, outdoor seating, parking, ro...",- the restaurant is too expensive and it’s not...
...,...,...,...,...,...,...,...,...
1438,Al Sayyadiyah Resturant,jeddah,saudi arabia,al sayyadiyah restaurant is one of the finest ...,arabic,"barbeque & grill, seafood, takeout","air conditioning, family seating, parking",very bad restaurant. no management. take time ...
1439,Al Sanabel Turkish Restaurant,madinah,saudi arabia,al sanabel restaurant is located in the hotel ...,"arabic, middle eastern, turkish","barbeque & grill, cafe, desserts, fine dining,...","air conditioning, family seating",worst ever experience there no food no milk fo...
1440,Lotus Restaurant,makkah,saudi arabia,lotus restaurant is one of the best restaurant...,international,"bakery, barbeque & grill, breakfast, coffee & ...","air conditioning, family seating, parking","it’s a very nice hotel to stay at, offering sp..."
1441,La Brasserie,jeddah,saudi arabia,la brasserie is a modern multi-cuisine restaur...,international,"bakery, barbeque & grill, breakfast, cafe, cof...","air conditioning, family seating, parking, wi-...","i stayed in this hotel more than 5 times, room..."


# Preprocess text data

In [419]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def process_sentences(text):
    temp_sent =[]

    # Tokenize words
    words = nltk.word_tokenize(text)

    # Lemmatize each of the words based on their position in the sentence
    tags = nltk.pos_tag(words)
    for i, word in enumerate(words):
        if tags[i][1] in ('VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'):  # only verbs
            lemmatized = lemmatizer.lemmatize(word, 'v')
        else:
            lemmatized = lemmatizer.lemmatize(word)

        # Remove stop words and non alphabet tokens
        if lemmatized not in stop_words and lemmatized.isalpha():
            temp_sent.append(lemmatized)

    # Some other clean-up
    full_sentence = ' '.join(temp_sent)
    full_sentence = full_sentence.replace("n't", " not")
    full_sentence = full_sentence.replace("'m", " am")
    full_sentence = full_sentence.replace("'s", " is")
    full_sentence = full_sentence.replace("'re", " are")
    full_sentence = full_sentence.replace("'ll", " will")
    full_sentence = full_sentence.replace("'ve", " have")
    full_sentence = full_sentence.replace("'d", " would")
    return full_sentence

In [420]:
sample_text = "I don't know where the cat went."
processed_text = process_sentences(sample_text)
print(processed_text)


I know cat go


In [421]:
df['reviews_processed'] = df['reviews'].apply(process_sentences)


In [422]:
df['style_processed'] = df['style'].apply(process_sentences)

In [423]:
df[['reviews', 'reviews_processed', 'style', 'style_processed']].sample(5)

,reviews,reviews_processed,style,style_processed
32025,"meatballs to die for!!, nice food. slightly ha...",meatball die nice food slightly haphazzard,"italian, mediterranean, vegetarian friendly",italian mediterranean vegetarian friendly
2400,"wouldn't recommend for orders, a good neighbor...",would recommend order good,italian,italian
11367,"simple and tasty go for the lebanese food, good",simple tasty go lebanese food good,"lebanese, mediterranean, middle eastern",lebanese mediterranean middle eastern
12061,"nice place with variety of choices, good food ...",nice place variety choice good food great beer,"german, central european, european",german central european european
34825,"very good home food, small but very good",good home food small good,"middle eastern, persian, halal",middle eastern persian halal


In [424]:
df['bag_of_words'] = df['style_processed'] + ' ' + df['reviews_processed']
display('Sample of bag_of_words attribute', df[['style', 'reviews' ,'bag_of_words']].sample(5))

'Sample of bag_of_words attribute'

,style,reviews,bag_of_words
27442,fast food,"oh so amazing, quick lunch if you're sick of t...",fast food oh amazing quick lunch sick tapa
31647,"thai, asian, vegetarian friendly, vegan options","fantastic, lovely food and personal services",thai asian vegetarian friendly vegan option fa...
41216,"chinese, asian, vegetarian friendly","the not so gourmet kitchen, reasonable food, r...",chinese asian vegetarian friendly gourmet kitc...
40299,"bar, british, pub, vegetarian friendly","busy pre gig drinks, the invsisble man",bar british pub vegetarian friendly busy pre g...
6654,"bar, mediterranean, european, spanish, vegetar...","average food, fun small nibbles",bar mediterranean european spanish vegetarian ...


In [425]:
df_saudi

,name,city,country,description,style,restaurant_type,restaurant_amenities,reviews
0,Thaiya,riyadh,saudi arabia,thaiya is a charming thai restaurant in riyadh...,thai,fine dining,"air conditioning, parking",creative take on thai cuisine. not very tradit...
1,Zafran Valley,riyadh,saudi arabia,zafran valley is a premium dining destination ...,indian,,,loved the delicious food at this lovely restau...
2,Okla,riyadh,saudi arabia,"mutah beale, a name hip-hop enthusiasts might ...",armenian,burger,"air conditioning, outdoor seating, parking","if you want a burger done right, this is the s..."
3,Gloria Cafe,jeddah,saudi arabia,"gloria café serves hot and cold drinks, fresh ...",,"breakfast, cafe, coffee & tea, desserts, fast ...","air conditioning, family seating, outdoor seat...","quiet and comfortable cafe?, it's a perfect ca..."
4,Nobu,jeddah,saudi arabia,nobu jeddah is the finest japanese restaurant ...,"japanese, peruvian","fine dining, seafood","air conditioning, outdoor seating, parking, ro...",- the restaurant is too expensive and it’s not...
...,...,...,...,...,...,...,...,...
1438,Al Sayyadiyah Resturant,jeddah,saudi arabia,al sayyadiyah restaurant is one of the finest ...,arabic,"barbeque & grill, seafood, takeout","air conditioning, family seating, parking",very bad restaurant. no management. take time ...
1439,Al Sanabel Turkish Restaurant,madinah,saudi arabia,al sanabel restaurant is located in the hotel ...,"arabic, middle eastern, turkish","barbeque & grill, cafe, desserts, fine dining,...","air conditioning, family seating",worst ever experience there no food no milk fo...
1440,Lotus Restaurant,makkah,saudi arabia,lotus restaurant is one of the best restaurant...,international,"bakery, barbeque & grill, breakfast, coffee & ...","air conditioning, family seating, parking","it’s a very nice hotel to stay at, offering sp..."
1441,La Brasserie,jeddah,saudi arabia,la brasserie is a modern multi-cuisine restaur...,international,"bakery, barbeque & grill, breakfast, cafe, cof...","air conditioning, family seating, parking, wi-...","i stayed in this hotel more than 5 times, room..."


In [426]:
df_saudi['reviews_processed'] = df_saudi['reviews'].apply(process_sentences)
df_saudi['style_processed'] = df_saudi['style'].apply(process_sentences)
df_saudi['description_processed'] = df_saudi['description'].apply(process_sentences)
df_saudi['restaurant_type_processed'] = df_saudi['restaurant_type'].apply(process_sentences)
df_saudi['restaurant_amenities_processed'] = df_saudi['restaurant_amenities'].apply(process_sentences)

In [427]:
df_saudi[['reviews', 'reviews_processed', 'style', 'style_processed',"description","description_processed","restaurant_type_processed","restaurant_type", "restaurant_amenities_processed","restaurant_amenities"]].sample(5)

,reviews,reviews_processed,style,style_processed,description,description_processed,restaurant_type_processed,restaurant_type,restaurant_amenities_processed,restaurant_amenities
43,this place is overrated. the indoor area is cr...,place overrate indoor area cramp smoking secti...,,,black potion is one of the best cafes in jedda...,black potion one best cafe jeddah place small ...,breakfast cafe coffee tea dessert fine dining ...,"breakfast, cafe, coffee & tea, desserts, fine ...",air conditioning family seating outdoor seatin...,"air conditioning, family seating, outdoor seat..."
822,"good foods\ngood place, the food was quite ave...",good food good place food quite average price ...,arabic,arabic,welcome to seeneez arabic restaurant! seeneez ...,welcome seeneez arabic restaurant seeneez offe...,barbeque grill breakfast coffee tea dessert fi...,"barbeque & grill, breakfast, coffee & tea, des...",air conditioning family seating park,"air conditioning, family seating, parking"
163,"nice cafe, good coffee, nice place ❤️, ❤, ?, i...",nice cafe good coffee nice place one biggest c...,,,welcome to solitaire world cafe. this place is...,welcome solitaire world cafe place new sun ray...,cafe coffee tea dessert fine dining takeout,"cafe, coffee & tea, desserts, fine dining, tak...",air conditioning family seating park,"air conditioning, family seating, parking"
75,"food was good alhamdulillaah.\nwe had pizza, p...",food good alhamdulillaah pizza pasta salmon ev...,italian,italian,valenti's is a fairly new italian restaurant w...,valenti fairly new italian restaurant locate a...,barbeque grill cafe dessert fine dining juice ...,"barbeque & grill, cafe, desserts, fine dining,...",air conditioning family seating parking internet,"air conditioning, family seating, parking, wi-..."
1421,the authentic chinese cuisine restaurant in ta...,authentic chinese cuisine restaurant taif grow...,chinese,chinese,china restaurant is a very traditional chinese...,china restaurant traditional chinese restauran...,fine dining,fine dining,air conditioning family seating,"air conditioning, family seating"


In [428]:
df_saudi.columns

Index(['name', 'city', 'country', 'description', 'style', 'restaurant_type',
       'restaurant_amenities', 'reviews', 'reviews_processed',
       'style_processed', 'description_processed', 'restaurant_type_processed',
       'restaurant_amenities_processed'],
      dtype='object')

In [429]:
df_saudi['bag_of_words'] = df_saudi['style_processed'] + ' ' + df_saudi['reviews_processed']+ ' ' + df_saudi["description_processed"]+ ' '+ df_saudi["restaurant_type_processed"]+ ' '+df_saudi["restaurant_amenities_processed"]
display('Sample of bag_of_words attribute', df_saudi[['style', 'reviews',"description" ,"restaurant_type","restaurant_amenities","bag_of_words"]].sample(5))

'Sample of bag_of_words attribute'

,style,reviews,description,restaurant_type,restaurant_amenities,bag_of_words
994,american,this our family's favorite burger house. try t...,the garage burgers serves great american food....,"fast food, takeout","air conditioning, family seating, outdoor seat...",american family favorite burger house try take...
877,italian,took chicken cheesy pasta\ntaste is good\nits ...,la casa pasta is famous for pasta. their pasta...,"coffee & tea, fine dining, juice bar, takeout","air conditioning, family seating, parking",italian take chicken cheesy pasta taste good c...
293,,"order via hunger station app , turkey sandwich...",50 degrees espresso house is a great place to ...,"cafe, coffee & tea, desserts, fine dining, jui...","air conditioning, family seating, outdoor seat...",order via hunger station app turkey sandwich ...
527,international,"loved the place, the atmosphere, the vibes…eve...",iris cafe is an elegant cafe and restaurant th...,"cafe, coffee & tea, desserts, fine dining",,international love place atmosphere point make...
115,,lily restaurant impresses with its diverse men...,lily restaurant is a lovely breakfast and brun...,breakfast,breakfast,lily restaurant impress diverse menu exceptio...


# combine the data 

In [430]:
df_saudi

,name,city,country,description,style,restaurant_type,restaurant_amenities,reviews,reviews_processed,style_processed,description_processed,restaurant_type_processed,restaurant_amenities_processed,bag_of_words
0,Thaiya,riyadh,saudi arabia,thaiya is a charming thai restaurant in riyadh...,thai,fine dining,"air conditioning, parking",creative take on thai cuisine. not very tradit...,creative take thai cuisine traditional unmista...,thai,thaiya charming thai restaurant riyadh offer a...,fine dining,air conditioning park,thai creative take thai cuisine traditional un...
1,Zafran Valley,riyadh,saudi arabia,zafran valley is a premium dining destination ...,indian,,,loved the delicious food at this lovely restau...,love delicious food lovely restaurant excellen...,indian,zafran valley premium dining destination riyad...,,,indian love delicious food lovely restaurant e...
2,Okla,riyadh,saudi arabia,"mutah beale, a name hip-hop enthusiasts might ...",armenian,burger,"air conditioning, outdoor seating, parking","if you want a burger done right, this is the s...",want burger right spot meat fresh juicy season...,armenian,mutah beale name enthusiast might recognize pa...,burger,air conditioning outdoor seating park,armenian want burger right spot meat fresh jui...
3,Gloria Cafe,jeddah,saudi arabia,"gloria café serves hot and cold drinks, fresh ...",,"breakfast, cafe, coffee & tea, desserts, fast ...","air conditioning, family seating, outdoor seat...","quiet and comfortable cafe?, it's a perfect ca...",quiet comfortable cafe perfect cafe guy watch ...,,gloria café serf hot cold drink fresh juice de...,breakfast cafe coffee tea dessert fast food fi...,air conditioning family seating outdoor seatin...,quiet comfortable cafe perfect cafe guy watch...
4,Nobu,jeddah,saudi arabia,nobu jeddah is the finest japanese restaurant ...,"japanese, peruvian","fine dining, seafood","air conditioning, outdoor seating, parking, ro...",- the restaurant is too expensive and it’s not...,restaurant expensive worth money tasty expect ...,japanese peruvian,nobu jeddah finest japanese restaurant jeddah ...,fine dining seafood,air conditioning outdoor seating parking roof ...,japanese peruvian restaurant expensive worth m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,Al Sayyadiyah Resturant,jeddah,saudi arabia,al sayyadiyah restaurant is one of the finest ...,arabic,"barbeque & grill, seafood, takeout","air conditioning, family seating, parking",very bad restaurant. no management. take time ...,bad restaurant management take time hour ready...,arabic,al sayyadiyah restaurant one finest luxurious ...,barbeque grill seafood takeout,air conditioning family seating park,arabic bad restaurant management take time hou...
1439,Al Sanabel Turkish Restaurant,madinah,saudi arabia,al sanabel restaurant is located in the hotel ...,"arabic, middle eastern, turkish","barbeque & grill, cafe, desserts, fine dining,...","air conditioning, family seating",worst ever experience there no food no milk fo...,worst ever experience food milk tea zero ambia...,arabic middle eastern turkish,al sanabel restaurant locate hotel name near m...,barbeque grill cafe dessert fine dining takeout,air conditioning family seating,arabic middle eastern turkish worst ever exper...
1440,Lotus Restaurant,makkah,saudi arabia,lotus restaurant is one of the best restaurant...,international,"bakery, barbeque & grill, breakfast, coffee & ...","air conditioning, family seating, parking","it’s a very nice hotel to stay at, offering sp...",nice hotel stay offer spotless room truly ambi...,international,lotus restaurant one best restaurant makkah in...,bakery barbeque grill breakfast coffee tea des...,air conditioning family seating park,international nice hotel stay offer spotless r...
1441,La Brasserie,jeddah,saudi arabia,la brasserie is a modern multi-cuisine restaur...,international,"bakery, barbeque & grill, breakfast, cafe, cof...","air conditioning, family seating, parking, wi-...","i st

In [431]:
df_saudi_2combine = df_saudi[['name', 'city', 'country', 'style', 'reviews', 'reviews_processed', 'style_processed', 'bag_of_words']]

In [433]:
df.shape

(76325, 8)

In [434]:
combined_data = pd.concat([df, df_saudi_2combine], ignore_index=True)
combined_data

,name,city,country,style,reviews,reviews_processed,style_processed,bag_of_words
0,Martine of Martine's Table,amsterdam,netherlands,"french, dutch, european","just like home, a warm welcome to wintry amste...",like home warm welcome wintry amsterdam,french dutch european,french dutch european like home warm welcome w...
1,De Silveren Spiegel,amsterdam,netherlands,"dutch, european, vegetarian friendly, gluten f...","great food and staff, just perfect",great food staff perfect,dutch european vegetarian friendly gluten free...,dutch european vegetarian friendly gluten free...
2,La Rive,amsterdam,netherlands,"mediterranean, french, international, european...","satisfaction, delicious old school restaurant",satisfaction delicious old school restaurant,mediterranean french international european ve...,mediterranean french international european ve...
3,Vinkeles,amsterdam,netherlands,"french, european, international, contemporary,...","true five star dinner, a superb evening of fin...",true five star dinner superb evening fine dini...,french european international contemporary veg...,french european international contemporary veg...
4,Librije's Zusje Amsterdam,amsterdam,netherlands,"dutch, european, international, vegetarian fri...","best meal.... ever, super food experience",best meal ever super food experience,dutch european international vegetarian friend...,dutch european international vegetarian friend...
...,...,...,...,...,...,...,...,...
77763,Al Sayyadiyah Resturant,jeddah,saudi arabia,arabic,very bad restaurant. no management. take time ...,bad restaurant management take time hour ready...,arabic,arabic bad restaurant management take time hou...
77764,Al Sanabel Turkish Restaurant,madinah,saudi arabia,"arabic, middle eastern, turkish",worst ever experience there no food no milk fo...,worst ever experience food milk tea zero ambia...,arabic middle eastern turkish,arabic middle eastern turkish worst ever exper...
77765,Lotus Restaurant,makkah,saudi arabia,international,"it’s a very nice hotel to stay at, offering sp...",nice hotel stay offer spotless room truly ambi...,international,international nice hotel stay offer spotless r...
77766,La Brasserie,jeddah,saudi arabia,international,"i stayed in this hotel more than 5 times, room...",stay hotel time room spacious classic furnitur...,international,international stay hotel time room spacious cl...


In [451]:
combined_data.columns

Index(['name', 'city', 'country', 'style', 'reviews', 'reviews_processed',
       'style_processed', 'bag_of_words'],
      dtype='object')

# Recommendation

In [456]:
def recommend(description):
    # Convert user input to lowercase
    description = description.lower()

    data = combined_data.copy()

    # # Extract cities
    cities_input = []
    for city in cities_list:
        if city in description:
            cities_input.append(city)
            description = description.replace(city, "")

    # if cities_input:
        data = data[data['city'].isin(cities_input)]

    # # Same thing for countries
    countries_input = []
    for country in countries_list:
        if country in description:
            countries_input.append(country)
            description = description.replace(country, "")

    # if countries_input:
        data = data[data['country'].isin(countries_input)]

    # Extract price class
    # for key, value in price_map.items():
    #     if any(v in description for v in value):
    #         data = data[data['price'] == key]
    #         break

    # Process user description text input
    description = process_sentences(description)
    description = description.strip()
    print('Processed user feedback:', description)

    # Init a TF-IDF vectorizer
    tfidfvec = TfidfVectorizer()

    # Fit data on processed reviews
    vec = tfidfvec.fit(data["bag_of_words"])
    features = vec.transform(data["bag_of_words"])

    # Transform user input data based on fitted model
    description_vector =  vec.transform([description])

    # Calculate cosine similarities between users processed input and reviews
    cos_sim = linear_kernel(description_vector, features)

    # Add similarities to data frame
    data['similarity'] = cos_sim[0]

    # Sort data frame by similarities
    data.sort_values(by='similarity', ascending=False, inplace=True)

    return data[['name', 'style', 'reviews', 'similarity']]

In [457]:
recommend("great dinner at a cozy restaurant in Riyadh.")

Processed user feedback: great dinner cozy restaurant riyadh


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [449]:
# Check for any empty strings in your text data
empty_reviews_rows = combined_data[combined_data['reviews_processed'].str.strip() == '']
print(empty_reviews_rows)


                                    name        city         country  \
2655                            De Quake   amsterdam     netherlands   
3830                Napoleon Fish Tavern      athens          greece   
6609                       221 Bar Tapas   barcelona           spain   
10643                             Barfly      berlin         germany   
12317        Marios Pizza Pasta by oliva      berlin         germany   
13474                         McDonald's  bratislava        slovakia   
16969               Loyola Cafe & Lounge    budapest         hungary   
17252           Royal Premium Restaurant    budapest         hungary   
26257                    Palacio de Buda      lisbon        portugal   
36961                            Yum yum      london  united kingdom   
36991                               Glo!      london  united kingdom   
39247                      Sushi O'Clock      london  united kingdom   
46751                             Tripea      madrid           s

In [450]:
empty_reviews_rows

,name,city,country,style,reviews,reviews_processed,style_processed,bag_of_words
2655,De Quake,amsterdam,netherlands,european,not again,,european,european
3830,Napoleon Fish Tavern,athens,greece,greek,ab_comments...,,greek,greek
6609,221 Bar Tapas,barcelona,spain,"french, mediterranean, spanish",'perfect',,french mediterranean spanish,french mediterranean spanish
10643,Barfly,berlin,germany,international,here we are!,,international,international
12317,Marios Pizza Pasta by oliva,berlin,germany,italian,!!!,,italian,italian
13474,McDonald's,bratislava,slovakia,"fast food, american","so and so, same same",,fast food american,fast food american
16969,Loyola Cafe & Lounge,budapest,hungary,"italian, international, european, hungarian, v...",so-so,,italian international european hungarian veget...,italian international european hungarian veget...
17252,Royal Premium Restaurant,budapest,hungary,"european, hungarian, international",child=140cm ?!?,,european hungarian international,european hungarian international
26257,Palacio de Buda,lisbon,portugal,"indian, vegetarian friendly",on,,indian vegetarian friendly,indian vegetarian friendly
36961,Yum yum,london,united kingdom,chinese,over-expectation,,chinese,chinese


In [445]:
combined_data[mask]

ValueError: Boolean array expected for the condition, not object

In [441]:
combined_data.isna().sum()

name                 0
city                 0
country              0
style                0
reviews              0
reviews_processed    0
style_processed      0
bag_of_words         0
dtype: int64